In [119]:
print("hello")

hello


In [1]:
import os
from dotenv import load_dotenv

#load enivronment variables from .env file
load_dotenv()

gorq_key= os.getenv("API_KEY")


In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
    groq_api_key=gorq_key

)
response=llm.invoke("what is the capital city of Texas")
print(response)

content='The capital city of Texas is Austin.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 42, 'total_tokens': 51, 'completion_time': 0.012, 'prompt_time': 0.002312491, 'queue_time': 0.195234993, 'total_time': 0.014312491}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_19ca34f5ed', 'finish_reason': 'stop', 'logprobs': None} id='run-8dcb7772-1a01-4ed7-b66a-1308fb045f1d-0' usage_metadata={'input_tokens': 42, 'output_tokens': 9, 'total_tokens': 51}


In [ ]:
#web scraping the job details
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://careers.nike.com/lead-data-scientist/job/R-56462")
job_posting_extracted= loader.load().pop().page_content


In [69]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """  
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the deatils about the job and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description' and if you have required and goot_to_have the dont mention them seperately as key instead make it as a list.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
"""
)

chain_extract = prompt_extract | llm
job_description_scraped = chain_extract.invoke(input={'page_data': job_posting_extracted})
print(job_description_scraped.content)

{
  "role": "Lead Data Scientist",
  "experience": "At least 3–5-years relevant working experience",
  "skills": [
    "Strong proficiency in programming languages Python and SQL",
    "Solid understanding of statistical analysis, data mining, and predictive modeling techniques",
    "Strong problem-solving skills and the ability to work independently and as part of a team",
    "Excellent communication and presentation skills"
  ],
  "description": "Develop, implement, deploy and maintain predictive models. Analyse large, complex datasets to extract insights and identify trends. Collaborate with stakeholders to understand business needs and provide data-driven solutions. Communicate findings and recommendations to non-technical stakeholders through visualizations and reports. Stay up to date with the latest industry trends and technologies in data science and machine/deep learning. Ensure data quality and integrity throughout the data lifecycle."
}


In [118]:
type(job_description_scraped.content)

str

In [71]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
job_description= json_parser.parse(job_description_scraped.content)
job_description

{'role': 'Lead Data Scientist',
 'experience': 'At least 3–5-years relevant working experience',
 'skills': ['Strong proficiency in programming languages Python and SQL',
  'Solid understanding of statistical analysis, data mining, and predictive modeling techniques',
  'Strong problem-solving skills and the ability to work independently and as part of a team',
  'Excellent communication and presentation skills'],
 'description': 'Develop, implement, deploy and maintain predictive models. Analyse large, complex datasets to extract insights and identify trends. Collaborate with stakeholders to understand business needs and provide data-driven solutions. Communicate findings and recommendations to non-technical stakeholders through visualizations and reports. Stay up to date with the latest industry trends and technologies in data science and machine/deep learning. Ensure data quality and integrity throughout the data lifecycle.'}

In [99]:
#reading the content from the resume
from docx import Document
import os

resume_doc= Document(os.path.join(os.getcwd(),  "Data-Scientist-Resume-Sample.docx"))
resume_context= [(para.text).strip() for para in resume_doc.paragraphs]
# print(resume_context)
resume_context_list= [content for content in resume_context if content]

In [100]:
from langchain.text_splitter import(
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter
)

character_splitter= RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n","."], chunk_size=500, chunk_overlap=50
)

character_split_text= character_splitter.split_text("\n\n".join(resume_context_list))

# print("chunk split",character_split_text)
# # print(word_wrap(character_split_text[10]))
print(f"\nTotal chunks: {len(character_split_text)}")


token_splitter= SentenceTransformersTokenTextSplitter(
    chunk_overlap=5, tokens_per_chunk=20
)
token_split_texts=[]
for text in character_split_text:
    token_split_texts+= token_splitter.split_text(text)
print(f"\nTotal token_splits: {(len(token_split_texts))}")
# print("token split")
# # print(token_split_texts)


Total chunks: 8

Total token_splits: 41


In [ ]:
import chromadb
from chromadb.utils import embedding_functions 

embedding_function= embedding_functions.SentenceTransformerEmbeddingFunction()

#instaitating chromadb client
chroma_client= chromadb.PersistentClient()

collection_name = "resume-context" 

# Check if it exists
if collection_name in chroma_client.list_collections():
    chroma_client.delete_collection(name=collection_name)
    print(f"Deleted existing collection: {collection_name}")

# Create a new one
chroma_collection = chroma_client.create_collection(
    name=collection_name,
    embedding_function=embedding_function
)



#extract the embeddings of the token_split_texts
ids=[str(i) for i in range(len(token_split_texts))]
chroma_collection.add( ids= ids, documents= token_split_texts)

Deleted existing collection: resume-context


In [ ]:
# query="what is are the skills that you have?"
# results= chroma_collection.query(query_texts=[query], n_results=5)
# retrived_documents= results["documents"][0]
# retrived_documents

['skills : probability & statistics, linear algebra, machine learning, computer science, artificial neural networks tools and',
 'ai ) simultaneously trying to synergies my skill sets of technological expertise and leadership qualities. i also',
 'leadership qualities. i also intend to devote most of my time in doing research and mark my significant contribution',
 'others : my core area of expertise in analysis and building predictive models. i have developed random forest',
 'card model. i am also fond of writing research papers. i have written a few papers related to']

In [107]:
candidate_name_retrived= chroma_collection.query(query_texts="what is the name of the person in the resume?", n_results=2)
candidated_name="".join(candidate_name_retrived["documents"][0])
candidated_name

'john doe data scientist summary career objective : i intend to explore a variety of areas in machine learning () - 7. 43 cpi2. bachelors – b. e. ( computer engineering - gujarat'

In [109]:
candidate_skills_retrived= chroma_collection.query(query_texts=job_description['skills'], n_results=5)
candidate_skills= "".join(candidate_skills_retrived["documents"][0])
candidate_skills

', hadoop, python, tableau past experience : 1. worked in an ai ( artificial intelligencemodeling, text analysis, java, ruby on rails, mysql, hadoop, pythonskills : probability & statistics, linear algebra, machine learning, computer science, artificial neural networks tools and, and analytics. java developer at quick developments, sqlyog, apache tomcat education nirma university, ahmedabad, gujarat, india master of'

In [114]:
prompt_email = PromptTemplate.from_template(
    """  
    ### JOB DESCRIPTION:
    {job_description}

    ### CANDIDATE skills:
    {candidate_skills}

    ### Candidate Name:
    {candidate_name}

    ### INSTRUCTION:
    Write a professional and short concise email to the hiring manager expressing interest in the above job. 
    start with the name and introduction of the candidate’s and highlight relevant skills and experience from the resume that align with the job description.
    The tone should be enthusiastic, confident, and tailored to the job.

    ### EMAIL (NO PREAMBLE):
    """
)


chain_email = prompt_email | llm
email_for_hiring_manager = chain_email.invoke({"job_description": str(job_description), "candidate_skills" : candidate_skills, "candidate_name" : candidated_name})

In [115]:
print(email_for_hiring_manager.content)

Subject: Application for Lead Data Scientist Position

Dear Hiring Manager,

I am writing to express my strong interest in the Lead Data Scientist position at your esteemed organization. My name is John Doe, and I am a seasoned data scientist with a passion for developing and implementing predictive models, analyzing complex datasets, and driving business growth through data-driven insights.

With over 5 years of experience in the field, I possess a strong proficiency in programming languages such as Python and SQL, as well as expertise in statistical analysis, data mining, and predictive modeling techniques. My experience with Hadoop, Java, and Ruby on Rails has also equipped me with the ability to work with large datasets and develop scalable solutions.

I am particularly drawn to this role because of the opportunity to collaborate with stakeholders, communicate complex findings to non-technical audiences, and stay up-to-date with the latest industry trends and technologies in data s